In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [75]:
#https://www.kaggle.com/c/digit-recognizer
data = np.genfromtxt('./train.csv', delimiter=',')

In [3]:
data = data[1:]

In [4]:
#Preparing Input Data
X = data[:,1:].astype(np.float32)
X = X/255

In [5]:
#Preparing Lablels Data
y = data[:,0:1]
enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(y)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=10)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, train_size=0.5, random_state=5)

In [68]:
n_input = 784
n_classes = 10
learning_rate = 0.00005
batch_size = 128
training_epochs = 40
n_hidden_layer_1 = 128
n_hidden_layer_2 = 256
dropout_prob_1 = 0.6
dropout_prob_2 = 0.8

In [69]:
weights = {
    'hidden_layer_1': tf.Variable(tf.random_normal([n_input, n_hidden_layer_1])),
    'hidden_layer_2': tf.Variable(tf.random_normal([n_hidden_layer_1, n_hidden_layer_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer_2, n_classes]))
}
baises = {
    'hidden_layer_1': tf.Variable(tf.random_normal([n_hidden_layer_1])),
    'hidden_layer_2': tf.Variable(tf.random_normal([n_hidden_layer_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [70]:
#MODEL
X_t = tf.placeholder(tf.float32, [None, n_input])
y_t = tf.placeholder(tf.float32, [None, n_classes])
keep_prob_1 = tf.placeholder('float')
keep_prob_2 = tf.placeholder('float')
hidden_layer_1 = tf.nn.relu(tf.add(tf.matmul(X_t, weights['hidden_layer_1']), baises['hidden_layer_1']))
hidden_layer_1 = tf.nn.dropout(hidden_layer_1, keep_prob_1)
hidden_layer_2 = tf.nn.relu(tf.add(tf.matmul(hidden_layer_1, weights['hidden_layer_2']), baises['hidden_layer_2']))
hidden_layer_2 = tf.nn.dropout(hidden_layer_2, keep_prob_2)
logits = tf.add(tf.matmul(hidden_layer_2, weights['out']), baises['out'])

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_t))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [71]:
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y_t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [72]:
saver = tf.train.Saver()

In [73]:
from sklearn.utils import shuffle
def split_into_batches(X, y, batch_size=128):
    assert X.shape[0] == y.shape[0]
    output_batches = []
    sample_size = int(X.shape[0])
    X_s, y_s = shuffle(X, y)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [X_s[start_i:end_i], y_s[start_i:end_i]]
        output_batches.append(batch)
    return output_batches

In [74]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Started Training')
    for epoch in range(training_epochs):
        batches = split_into_batches(X_train, y_train, batch_size=batch_size)
        for [batch_X, batch_y] in batches:
            sess.run(optimizer, feed_dict={X_t: batch_X, y_t: batch_y, keep_prob_1: dropout_prob_1, keep_prob_2: dropout_prob_2})
        valid_accuracy = sess.run(accuracy, feed_dict={X_t: X_val, y_t: y_val, keep_prob_1: 1.0, keep_prob_2: 1.0})
        training_accuracy = sess.run(accuracy, feed_dict={X_t: X_train, y_t: y_train, keep_prob_1: 1.0, keep_prob_2: 1.0})
        print('Epoch {:<3} - Validation accuracy: {} Training accuracy {}'.format(epoch, valid_accuracy, training_accuracy))
    saver.save(sess, './mnist.ckpt')

Started Training
Epoch 0   - Validation accuracy: 0.12904761731624603 Training accuracy 0.1405952423810959
Epoch 1   - Validation accuracy: 0.17499999701976776 Training accuracy 0.18211309611797333
Epoch 2   - Validation accuracy: 0.2202380895614624 Training accuracy 0.22630952298641205
Epoch 3   - Validation accuracy: 0.2669047713279724 Training accuracy 0.275773823261261
Epoch 4   - Validation accuracy: 0.31857141852378845 Training accuracy 0.3288392722606659
Epoch 5   - Validation accuracy: 0.3697619140148163 Training accuracy 0.37788689136505127
Epoch 6   - Validation accuracy: 0.4166666567325592 Training accuracy 0.4263392984867096
Epoch 7   - Validation accuracy: 0.4642857015132904 Training accuracy 0.470714271068573
Epoch 8   - Validation accuracy: 0.4985714256763458 Training accuracy 0.5092262029647827
Epoch 9   - Validation accuracy: 0.528333306312561 Training accuracy 0.5406547784805298
Epoch 10  - Validation accuracy: 0.5573809742927551 Training accuracy 0.5685119032859802
E

In [62]:
with tf.Session() as sess:
    saver.restore(sess, './mnist.ckpt')
    print(sess.run(logits, feed_dict={X_t: X_test[0:1], keep_prob_1: 1.0, keep_prob_2: 1.0}))

[[-149.26896667  517.29394531  118.0597229    88.37108612   26.54593277
    -8.07290077 -178.22814941  -81.26624298  -29.87822151  -15.71733761]]


In [59]:
with tf.Session() as sess:
    saver.restore(sess, './mnist.ckpt')
    print(sess.run(logits, feed_dict={X_t: X_test[0:1], keep_prob_1: 1.0, keep_prob_2: 1.0}))

[[-315.43103027  688.57531738   86.82846832  167.62110901 -719.04998779
   207.35830688 -196.04437256 -182.35073853   46.22846985 -147.48817444]]
